In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from typing import List, Tuple
import os
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data
print("📂 Loading processed data...")
train_df = pd.read_pickle('../data/processed/train.pkl')
test_df = pd.read_pickle('../data/processed/test.pkl')

# Variants
variants = [
    'without_lemma',
    'with_lemma',
    'with_lemma_pos',
    'with_dep_tree',
    'with_chunking'
]

📂 Loading processed data...


In [3]:
# Function to prepare text from tokens/ngrams
def prepare_text(tokens: List, ngrams_b: List[Tuple], ngrams_t: List[Tuple], use_ngrams: bool = False) -> str:
    """
    Convert tokens and optionally ngrams to string for TF-IDF.
    For ngrams, join tuples into space-separated strings.
    """
    if not tokens:
        return ''
    # Handle different token formats (str or tuples)
    if isinstance(tokens[0], str):
        text = ' '.join(tokens)
    elif isinstance(tokens[0], tuple):
        text = ' '.join(['_'.join(t) for t in tokens])
    else:
        text = ''
    
    if use_ngrams:
        bigrams_str = ' '.join(['_'.join(gram) for gram in ngrams_b]) if ngrams_b else ''
        trigrams_str = ' '.join(['_'.join(gram) for gram in ngrams_t]) if ngrams_t else ''
        text = f"{text} {bigrams_str} {trigrams_str}".strip()
    
    return text

In [4]:
# Results storage
results_multi = {}
results_binary = {dim: {} for dim in ['IE', 'NS', 'FT', 'JP']}

In [6]:
# Loop over variants
for var in variants:
    print(f"\n🔍 Processing variant: {var}")
    
    # Prepare train and test texts
    print("📝 Preparing text features...")
    tqdm.pandas()
    train_df['text'] = train_df.progress_apply(
        lambda row: prepare_text(row[f'tokens_{var}'], row[f'Bigrams_{var}'], row[f'Trigrams_{var}'], use_ngrams=True), axis=1
    )
    test_df['text'] = test_df.progress_apply(
        lambda row: prepare_text(row[f'tokens_{var}'], row[f'Bigrams_{var}'], row[f'Trigrams_{var}'], use_ngrams=True), axis=1
    )
    
    X_train = train_df['text']
    X_test = test_df['text']
    
    # Multi-class (16 types)
    print("🧠 Training multi-class model with balanced class weights...")
    y_train_multi = train_df['type']
    y_test_multi = test_df['type']
    
    pipeline_multi = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,3))),
        ('clf', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, class_weight='balanced'))
    ])
    # params = {'clf__C': [0.1, 1, 10], 'clf__solver': ['lbfgs', 'liblinear']}
    # grid = GridSearchCV(pipeline_multi, params, cv=5, scoring='f1_weighted')
    # grid.fit(X_train, y_train_multi)
    
    pipeline_multi.fit(X_train, y_train_multi)
    
    y_pred_multi = pipeline_multi.predict(X_test)
    
    acc_multi = accuracy_score(y_test_multi, y_pred_multi)
    f1_multi = f1_score(y_test_multi, y_pred_multi, average='weighted')
    
    results_multi[var] = {'accuracy': acc_multi, 'f1': f1_multi}
    print(f"Multi-class - Accuracy: {acc_multi:.4f}, F1: {f1_multi:.4f}")
    print(classification_report(y_test_multi, y_pred_multi))
    
    # Save model
    os.makedirs('models', exist_ok=True)
    joblib.dump(pipeline_multi, f'models/multi_{var}.pkl')
    
    # Binary classifiers for each dimension
    for dim in ['IE', 'NS', 'FT', 'JP']:
        print(f"🧠 Training binary model for {dim} with balanced class weights...")
        y_train_bin = train_df[dim]
        y_test_bin = test_df[dim]
        
        pipeline_bin = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,3))),
            ('clf', LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced'))
        ])
        
        pipeline_bin.fit(X_train, y_train_bin)
        y_pred_bin = pipeline_bin.predict(X_test)
        
        acc_bin = accuracy_score(y_test_bin, y_pred_bin)
        f1_bin = f1_score(y_test_bin, y_pred_bin, average='weighted')
        
        results_binary[dim][var] = {'accuracy': acc_bin, 'f1': f1_bin}
        print(f"{dim} - Accuracy: {acc_bin:.4f}, F1: {f1_bin:.4f}")
        print(classification_report(y_test_bin, y_pred_bin))
        
        # Save model
        joblib.dump(pipeline_bin, f'models/binary_{dim}_{var}.pkl')


🔍 Processing variant: without_lemma
📝 Preparing text features...


100%|██████████| 1735/1735 [00:00<00:00, 2654.68it/s]


🧠 Training multi-class model with balanced class weights...
Multi-class - Accuracy: 0.3527, F1: 0.3671
              precision    recall  f1-score   support

        ENFJ       0.14      0.32      0.19        38
        ENFP       0.35      0.40      0.37       135
        ENTJ       0.10      0.24      0.14        46
        ENTP       0.38      0.39      0.38       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.04      0.06      0.05        18
        INFJ       0.48      0.26      0.34       294
        INFP       0.56      0.45      0.50       366
        INTJ       0.37      0.30      0.33       218
        INTP       0.50      0.41      0.45       261
        ISFJ       0.17      0.36      0.23        33
        ISFP       0.17      0.31      0.22        54
        ISTJ       0.14      0.20      0.16        41
        ISTP       0.26      0.4

100%|██████████| 1735/1735 [00:00<00:00, 2026.20it/s]


🧠 Training multi-class model with balanced class weights...
Multi-class - Accuracy: 0.3470, F1: 0.3608
              precision    recall  f1-score   support

        ENFJ       0.12      0.26      0.17        38
        ENFP       0.33      0.38      0.35       135
        ENTJ       0.12      0.26      0.17        46
        ENTP       0.35      0.39      0.37       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.04      0.06      0.05        18
        INFJ       0.49      0.25      0.33       294
        INFP       0.56      0.46      0.50       366
        INTJ       0.35      0.27      0.30       218
        INTP       0.49      0.41      0.45       261
        ISFJ       0.13      0.24      0.17        33
        ISFP       0.14      0.28      0.18        54
        ISTJ       0.15      0.22      0.18        41
        ISTP       0.29      0.5

...

🧠 Training multi-class model with balanced class weights...
Multi-class - Accuracy: 0.3412, F1: 0.3572
              precision    recall  f1-score   support

        ENFJ       0.10      0.21      0.14        38
        ENFP       0.29      0.35      0.32       135
        ENTJ       0.11      0.26      0.15        46
        ENTP       0.37      0.39      0.38       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.04      0.06      0.05        18
        INFJ       0.50      0.27      0.35       294
        INFP       0.54      0.43      0.48       366
        INTJ       0.35      0.29      0.32       218
        INTP       0.49      0.40      0.44       261
        ISFJ       0.12      0.24      0.16        33
        ISFP       0.15      0.30      0.20        54
        ISTJ       0.18      0.27      0.22        41
        ISTP       0.26      0.4

100%|██████████| 1735/1735 [00:00<00:00, 5677.93it/s]


🧠 Training multi-class model with balanced class weights...
Multi-class - Accuracy: 0.1516, F1: 0.1692
              precision    recall  f1-score   support

        ENFJ       0.04      0.11      0.05        38
        ENFP       0.19      0.24      0.21       135
        ENTJ       0.04      0.13      0.06        46
        ENTP       0.18      0.18      0.18       137
        ESFJ       0.08      0.11      0.09         9
        ESFP       0.04      0.10      0.06        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.00      0.00      0.00        18
        INFJ       0.28      0.13      0.17       294
        INFP       0.34      0.16      0.22       366
        INTJ       0.23      0.14      0.17       218
        INTP       0.26      0.13      0.18       261
        ISFJ       0.04      0.12      0.06        33
        ISFP       0.07      0.20      0.11        54
        ISTJ       0.05      0.12      0.07        41
        ISTP       0.09      0.1

100%|██████████| 1735/1735 [00:00<00:00, 2331.27it/s]


🧠 Training multi-class model with balanced class weights...
Multi-class - Accuracy: 0.2767, F1: 0.2926
              precision    recall  f1-score   support

        ENFJ       0.08      0.21      0.12        38
        ENFP       0.28      0.33      0.30       135
        ENTJ       0.10      0.30      0.16        46
        ENTP       0.31      0.28      0.29       137
        ESFJ       0.05      0.11      0.06         9
        ESFP       0.05      0.10      0.07        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.00      0.00      0.00        18
        INFJ       0.37      0.19      0.25       294
        INFP       0.48      0.34      0.39       366
        INTJ       0.28      0.20      0.24       218
        INTP       0.45      0.35      0.39       261
        ISFJ       0.11      0.27      0.16        33
        ISFP       0.12      0.24      0.16        54
        ISTJ       0.14      0.22      0.17        41
        ISTP       0.23      0.4

In [8]:
# Compare results
print("\n📊 Comparison of Multi-class Results:")
for var, res in results_multi.items():
    print(f"{var}: Accuracy={res['accuracy']:.4f}, F1={res['f1']:.4f}")

for dim in ['IE', 'NS', 'FT', 'JP']:
    print(f"\n📊 Comparison of Binary {dim} Results:")
    for var, res in results_binary[dim].items():
        print(f"{var}: Accuracy={res['accuracy']:.4f}, F1={res['f1']:.4f}")

print("✅ Training and evaluation complete! Models saved in models/ directory.")


📊 Comparison of Multi-class Results:
without_lemma: Accuracy=0.3527, F1=0.3671
with_lemma: Accuracy=0.3470, F1=0.3608
with_lemma_pos: Accuracy=0.3412, F1=0.3572
with_dep_tree: Accuracy=0.1516, F1=0.1692
with_chunking: Accuracy=0.2767, F1=0.2926

📊 Comparison of Binary IE Results:
without_lemma: Accuracy=0.7308, F1=0.7447
with_lemma: Accuracy=0.7297, F1=0.7436
with_lemma_pos: Accuracy=0.7222, F1=0.7368
with_dep_tree: Accuracy=0.6571, F1=0.6775
with_chunking: Accuracy=0.7037, F1=0.7196

📊 Comparison of Binary NS Results:
without_lemma: Accuracy=0.7856, F1=0.8067
with_lemma: Accuracy=0.7844, F1=0.8053
with_lemma_pos: Accuracy=0.7850, F1=0.8061
with_dep_tree: Accuracy=0.6928, F1=0.7291
with_chunking: Accuracy=0.7556, F1=0.7813

📊 Comparison of Binary FT Results:
without_lemma: Accuracy=0.7821, F1=0.7824
with_lemma: Accuracy=0.7879, F1=0.7882
with_lemma_pos: Accuracy=0.7775, F1=0.7779
with_dep_tree: Accuracy=0.6496, F1=0.6501
with_chunking: Accuracy=0.7487, F1=0.7490

📊 Comparison of Binar